In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [4]:
spark = SparkSession.builder.appName("explore").getOrCreate()


24/10/08 20:16:56 WARN Utils: Your hostname, codespaces-a68b72 resolves to a loopback address: 127.0.0.1; using 10.0.15.132 instead (on interface eth0)
24/10/08 20:16:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 20:16:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/08 20:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
import os

movie_tables = {}

for file in os.listdir("../data"):
    print(file.split(".")[:2])
    movie_tables["_".join(file.split(".")[:2])] = (
        spark.read
        .options(**{
            "sep": "\t",
            "header": True,
            "compression": "gzip",
            "nullValue": r"\N",
        })
        .csv(f"../data/{file}")
    )
    # display(pd.read_csv(f"../data/{file}", compression = "gzip", sep = "\t", nrows = 10, na_values=r"\N").head(10))

['title', 'akas']
['title', 'principals']
['title', 'ratings']
['title', 'basics']
['name', 'basics']
['title', 'episode']
['title', 'crew']


24/10/08 20:17:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
import metadata.title_akas


metadata.title_akas.schema

StructType([StructField('titleId', StringType(), True), StructField('ordering', IntegerType(), True), StructField('title', StringType(), True), StructField('region', StringType(), True), StructField('language', StringType(), True), StructField('types', ArrayType(StringType(), True), True), StructField('attributes', ArrayType(StringType(), True), True), StructField('isOriginalTitle', BooleanType(), True)])

In [7]:
def when_mapping(column, _dict: dict) -> pyspark.sql.Column:
    base_output = F
    for cond, val in _dict.items():
        base_output = base_output.when(column == cond, val)

    return base_output

In [11]:
movie_tables["title_akas"].select(
 F.col("titleId"),
 F.col("ordering").cast(T.IntegerType()),
 F.col("title"),
 F.col("region"),
 F.col("language"),
 F.col("types"),
 F.col("attributes"),
 when_mapping(F.col("isOriginalTitle"), {1: True,  0: False}).alias("isOriginalTitle"),
).to(metadata.title_akas.schema).printSchema()


AnalysisException: [INVALID_COLUMN_OR_FIELD_DATA_TYPE] Column or field `types` is of type "STRING" while it's required to be "ARRAY<STRING>".